This code will create a STAC catalog and add several items from a local folder

How do I put the date of image acquisition for an imagery product?  I don't think there is an automated way to do it. I need to manually input it. But how? Maybe I need to add metadata to imagery products within Cyverse and then read that metadata with pystack? How do I write metadata in Cyverse? 

In [1]:
%pip install rasterio shapely pystac

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path
import pystac
import os

catalog = pystac.Catalog(
    id='Cyverse Remotely Sensed Imagery STAC Catalog',
    description='This catalog includes all of the imagery assets the exist in Cyverse Data Store',
    stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json']
)

In [3]:
from shapely.geometry import Polygon, mapping
import rasterio


def get_bbox_and_footprint(dataset):

    # create the bounding box it will depend if it comes from rasterio or rioxarray
    bounds = dataset.bounds

    if isinstance(bounds, rasterio.coords.BoundingBox):
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
    else:
        bbox = [float(f) for f in bounds()]

    # create the footprint
    footprint = Polygon([
        [bbox[0], bbox[1]],
        [bbox[0], bbox[3]],
        [bbox[2], bbox[3]],
        [bbox[2], bbox[1]]
    ])

    return bbox, mapping(footprint)

    return bbox, mapping(footprint)

In [4]:

#from rasterio.warp import transform_bounds, transform_geom


# get a list of files
folder = Path('/data-store/iplant/home/jgillan/STAC_drone')
masks = list(folder.rglob('*.tif'))

for mask in masks:

    # open the mask with rasterio
    ds = rasterio.open(mask)
    
    #Get raster's CRS
    raster_crs = ds.crs
    
    
    bbox, footprint = get_bbox_and_footprint(ds)
    
    idx = mask.stem
    
    from datetime import datetime
    
   #Define date and time
    year = 2019
    month = 5
    day = 25
    hour = 12
    
    #Create datetime object
    user_datetime = datetime(year, month, day, hour)
    
    
    
    item = pystac.Item(id=idx,
                 geometry=footprint,
                 bbox=bbox,
                 datetime=user_datetime,
                 properties={})
        
    
    catalog.add_item(item)

print(len(list(catalog.get_items())))
catalog.describe()

3
* <Catalog id=Cyverse Remotely Sensed Imagery STAC Catalog>
  * <Item id=1a_g2_ortho>
  * <Item id=1a_g2_sept_CHM>
  * <Item id=1a_u6_ortho>


In [5]:
from pystac.extensions.projection import AssetProjectionExtension

for mask in masks:
    idx = mask.stem
    item = catalog.get_item(idx)

    # as before, let's open the mask with rasterio and get bbox and footprint
    ds = rasterio.open(mask)
    bbox, footprint = get_bbox_and_footprint(ds)

    item.add_asset(
        key='blahblah',
        asset=pystac.Asset(
            href=mask.as_posix(),
            media_type=pystac.MediaType.GEOTIFF
        )
    )

In [6]:
stac_directory="/data-store/iplant/home/jgillan/STAC_drone"

catalog.normalize_hrefs(os.path.join(stac_directory, 'stac_test7'))

catalog.make_all_asset_hrefs_relative()
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)